In [9]:
# get_ipython().magic('matplotlib notebook')
# get_ipython().magic('matplotlib inline')
# get_ipython().magic('load_ext autoreload')
# get_ipython().magic('autoreload 2')
%matplotlib inline
%load_ext autoreload
%autoreload 2
    
#___________________________________________________________________________________________________________________
import os
import tripyview as tpv
import shapefile as shp
import numpy     as np
import xarray    as xr
import time      as clock
import matplotlib.pyplot as plt
import warnings
xr.set_options(keep_attrs=True)
do_parallel       = False
parallel_nprc     = 48   # number of dask workers
parallel_nprc_bin = 10   # number of processor used to parallize the binning loop
parallel_tmem     = 200  # max. available RAM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### INFO:
To compute especially on large unstructured grids #vertices>1M, you need to run this notebook in parallel (do_parallel=True) on several workers (parallel_nprc...is the number of dask worker that can be allocated, parallel_tmem...is the maximum available RAM that will be distributed between the dask workers). Therefor allocate a full !!! COMPUTE NODE !!! (admins might not be happy if you do this in parallel on a login node) of a HPC of your choice with as much memory (RAM) as you can get to run this notebook!

In [10]:
if do_parallel:
    from dask.distributed import Client
    # from dask.diagnostics import ProgressBar
    import dask
    print(' --> memory_limit: {:3.3f} GB'.format(parallel_tmem/(parallel_nprc)))
    ## dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
    client = Client(n_workers=parallel_nprc, threads_per_worker=1, memory_limit='{:3.3f} GB'.format(parallel_tmem/parallel_nprc))
    client

In [11]:
# Parameters
mesh_path = '/albedo/work/user/pscholz/mesh_fesom2.0/core2_srt_dep@node/'
# mesh_path = '/albedo/work/user/pscholz/mesh_fesom2.0/dart_test/'
save_path = None #'~/figures/test_papermill/'
save_fname= None

#_____________________________________________________________________________________
which_cycl= None #5 # set None --> take path as in input_paths otherwise add dir of cycle
which_mode= 'mhflx' # global heat flux

#_____________________________________________________________________________________
input_paths= list()
# input_paths.append('/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke+idemix_jayne_bin_ck0.1/5/')
# input_paths.append('/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke+idemix_nycander_bin_ck0.1/')
# input_paths.append('/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke+idemix_stormtide_bin_ck0.1/')
input_paths.append('/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke_ctrl_ck0.1/5/')
# input_paths.append('/albedo/work/user/pscholz/results/dart_linfs_pc0_ctrl_1/1/')

input_names= list()
# input_names.append('TKE+IDEMIX, jayne')
# input_names.append('TKE+IDEMIX, nycander')
# input_names.append('TKE+IDEMIX, stormtide')
input_names.append('TKE')

vname     = 'mhflx'
year      = [1979,2019]
mon, day, record, box, depth = None, None, None, None, None

#_____________________________________________________________________________________
box_region = list()
box_region.append('global')
box_region.append('moc_basins/Atlantic_MOC.shp')
box_region.append('moc_basins/IndoPacific_MOC.shp')
# box_region.append('ocean_basins/Atlantic_Basin.shp')
# box_region.append('ocean_basins/Pacific_Basin.shp')

#_____________________________________________________________________________________
# do anomaly plots in case ref_path is not None
ref_path  = None #'/home/ollie/pscholz/results/trr181_tke_ctrl_ck0.1/' # None
ref_name  = None # 'TKE, ck=0.1' # None
ref_year  = None # [2009,2019]
ref_mon, ref_day, ref_record = None, None, None

#_____________________________________________________________________________________
cstr, cnum = 'blue2red', 20
cref, crange, cmin, cmax, cfac, climit = None, None, None, None, None, None
chist, ctresh = True, 0.995

#_____________________________________________________________________________________
which_dpi = 300
do_allcycl= False
use_advflx= True  # True: compute hflux from u*t and v*t, False: compute hflx from u, v, temp,+(u_bolus, v_bolus)
use_bolusv= True  # True: include bolus velocity
do_edgevec_r2g = False

In [12]:
#___LOAD FESOM2 MESH___________________________________________________________________________________
mesh=tpv.load_mesh_fesom2(mesh_path, do_rot='None', focus=0, do_info=True, do_pickle=True,
                      do_earea=True, do_narea=True)

#______________________________________________________________________________________________________
if (which_cycl is not None) and (which_cycl != "None"): 
    #__________________________________________________________________________________________________
    if do_allcycl: cycl_s=1
    else         : cycl_s=which_cycl    
    
    #__________________________________________________________________________________________________
    aux_path, aux_name = list(), list()
    input_paths_old, input_names_old = input_paths, input_names
    for ii, (ipath,iname) in enumerate(zip(input_paths,input_names)):
        for ii_cycl in range(cycl_s, which_cycl+1):
            # input_paths[ii] = os.path.join(ipath,'{:d}/'.format(which_cycl))
            aux_path.append(os.path.join(ipath,'{:d}/'.format(ii_cycl)))
            if not do_allcycl: aux_name.append('{}'.format(iname))
            else             : aux_name.append('{:d}) {}'.format(ii_cycl, iname))
            print(ii, aux_path[-1],aux_name[-1])
    input_paths, input_names = aux_path, aux_name
    
    #__________________________________________________________________________________________________
    if (ref_path is not None) and (ref_path != "None"): 
        aux_path, aux_name = list(), list()
        ref_path_old, ref_name_old = ref_path, ref_name
        for ii_cycl in range(cycl_s, which_cycl+1):
            #ref_path = os.path.join(ref_path,'{:d}/'.format(which_cycl))
            aux_path.append(os.path.join(ref_path,'{:d}/'.format(ii_cycl)))
            if not do_allcycl: aux_name.append('{}'.format(ref_name))
            else             : aux_name.append('{:d}) {}'.format(ii_cycl, ref_name))
            print('R', ref_path[-1])        
        ref_path, ref_name = aux_path, aux_name
    del(aux_path, aux_name)
    
#______________________________________________________________________________________________________
cinfo=tpv.set_cinfo(cstr, cnum, crange, cmin, cmax, cref, cfac, climit, chist, ctresh)
ref_cinfo=None
if (ref_path is not None) and (ref_path != "None"): 
    if ref_year   is None: ref_year   = year
    if ref_mon    is None: ref_mon    = mon
    if ref_record is None: ref_record = record
    cinfo['cref']=0.0 
    ref_cinfo=tpv.set_cinfo(ref_cstr, ref_cnum, ref_crange, ref_cmin, ref_cmax, ref_cref, ref_cfac, ref_climit, ref_chist, ref_ctresh)
else:
    do_reffig=False

#______________________________________________________________________________________________________    
# concatenate list = list1+list2
if (ref_path is not None) and (ref_path != "None"): 
    if isinstance(ref_path, list): 
        input_paths, input_names = input_paths + ref_path, input_names + ref_name 
    else:    
        input_paths, input_names = input_paths + list([ref_path]), input_names + list([ref_name])
        
#______________________________________________________________________________________________________
box = list()
shp_path = os.path.join(tpv.__path__[0],'shapefiles/')
for region in box_region:
    if region == 'global' or isinstance(region,list): box.append(region)
    else: box.append(shp.Reader(os.path.join(shp_path,region)))        

#______________________________________________________________________________________________________
# use number of worker dependent chunk size for nodes and elements
chunks = dict({'time' : 'auto', 
               'elem' : 'auto', 
               'nod2' : 'auto', 
               'nz1'  : 'auto', 
               'nz'   : 'auto',
               'edg_n': 'auto'})

 > found *.pckl file: /albedo/work/user/pscholz/mesh_fesom2.0/core2_srt_dep@node
 > load  *.pckl file: tripyview_fesom2_core2_srt_dep@node_focus0.pckl
___FESOM2 MESH INFO________________________
 > path            = /work/ollie/pscholz/mesh_fesom2.0/core2_srt_dep@node
 > id              = core2_srt_dep@node
 > do rot          = None
 > [al,be,ga]      = 50, 15, -90
 > do augmpbnd     = True
 > do cavity       = False
 > do lsmask       = True
 > do earea,eresol = True, False
 > do narea,nresol = True, False
___________________________________________
 > #node           = 126858
 > #elem           = 244659
 > #lvls           = 48
___________________________________________


/albedo/home/pscholz/tripyview/tripyview/sub_mesh.py:139: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  mesh = pickle.load(fid)


In [13]:
ts = clock.time()
datapath = input_paths[0]
#___________________________________________________________________________
# load information about edges 
fname = 'fesom.mesh.diag.nc'
# check for directory with diagnostic file
if   os.path.isfile( os.path.join(datapath, fname) ): 
    dname = datapath
elif os.path.isfile( os.path.join( os.path.join(os.path.dirname(os.path.normpath(datapath)),'1/'), fname) ): 
    dname = os.path.join(os.path.dirname(os.path.normpath(datapath)),'1/')
elif os.path.isfile( os.path.join(mesh.path,fname) ): 
    dname = mesh.path
else:
    raise ValueError('could not find directory with...mesh.diag.nc file')    

#___________________________________________________________________________
# set specfic type when loading --> #convert to specific precision
# drop unnecessary variables:  Based on the documentation (but a bit hidden), 
# the "data_vars=" argument only works with Python 3.9.
from functools import partial
var_keep = ['edges', 'edge_tri', 'edge_cross_dxdy']
def _preprocess(x):
    for var in list(x.keys()):
        if var not in var_keep: 
            x = x.drop_vars(var)
            continue
        if x[var].dtype=='float64': x[var] = x[var].astype('float32')
    return x
partial_func = partial(_preprocess)

#___________________________________________________________________________
# load diag file --> apply drop variables by preprocessor function
mdiag = xr.open_mfdataset(os.path.join(dname,fname), parallel=False, 
                             chunks=dict({'edg_n':'auto'}), engine='netcdf4', 
                             preprocess=partial_func)
mdiag = mdiag.drop_vars(list(mdiag.coords)).unify_chunks()

# node indices of edge points [2 x n2ded]--> indices in python start with 0
mdiag['edges']    = mdiag['edges']-1
# element indices of triangles that are left and right of edg: [2 x n2ded]
# --> indices in python start with 0
mdiag['edge_tri'] = mdiag['edge_tri']-1

# Suppress the specific warning about sending large graphs
warnings.filterwarnings("ignore", category=UserWarning, message="Sending large graph of size")

#___________________________________________________________________________
# add edge coordinates and edge mid point coordinates
set_mdiag_chunk = dict({'edg_n': mdiag.chunksizes['edg_n']})
mdiag = mdiag.assign(edge_x = xr.DataArray(mesh.n_x[mdiag.edges], dims=['n2','edg_n']).astype('float32').chunk(set_mdiag_chunk))
mdiag = mdiag.assign(edge_y = xr.DataArray(mesh.n_y[mdiag.edges], dims=['n2','edg_n']).astype('float32').chunk(set_mdiag_chunk))
# mdiag = mdiag.drop_vars('edges')

mdiag = mdiag.assign(edge_mx = mdiag.edge_x.sum(dim='n2')/2.0)
mdiag = mdiag.assign(edge_my = mdiag.edge_y.sum(dim='n2')/2.0)

#___________________________________________________________________________
# Be sure that the edge_cross_dxdy variable is in the same rotational frame as your velocities. By default
# edge_cross_dxdy is in rotated coordinates. So if you velocities are also in rotated coordinates things are fine.
# If your velocities should be in geo coordinates than edge_cross_dxdy needs to be rotated as well into geo
# coordinates (do_edgevec_r2g=True)
if (do_edgevec_r2g):
    mdiag.edge_cross_dxdy[0,:], mdiag.edge_cross_dxdy[1,:] = tpv.vec_r2g(mesh.abg, mdiag.edge_mx, mdiag.edge_my, 
                                                                         mdiag.edge_cross_dxdy[0,:], mdiag.edge_cross_dxdy[1,:], 
                                                                         gridis='geo', do_info=False )
    mdiag.edge_cross_dxdy[2,:], mdiag.edge_cross_dxdy[3,:] = tpv.vec_r2g(mesh.abg, mdiag.edge_mx, mdiag.edge_my, 
                                                                         mdiag.edge_cross_dxdy[2,:], mdiag.edge_cross_dxdy[3,:], 
                                                                         gridis='geo', do_info=False )

#___________________________________________________________________________
set_mdiag_chunk = dict({'edg_n': mdiag.chunksizes['edg_n'], 'n2':mdiag.chunksizes['n2']})
# dx & dy of left triangle --> norm vector 
mdiag = mdiag.assign(edge_dxdy_l=xr.DataArray( np.array([ mdiag.edge_cross_dxdy[1,:], 
                                                         -mdiag.edge_cross_dxdy[0,:]]), dims=['n2', 'edg_n']).chunk(set_mdiag_chunk))
# dx & dy of right triangle --> norm vector 
mdiag = mdiag.assign(edge_dxdy_r=xr.DataArray( np.array([-mdiag.edge_cross_dxdy[3,:], 
                                                          mdiag.edge_cross_dxdy[2,:]]), dims=['n2', 'edg_n']).chunk(set_mdiag_chunk))
# if boundarie edge --> right triangle doesnot exist --> therefor dx,dy is zero there
mdiag.edge_dxdy_r[:, mdiag.edge_tri[1,:]<0]=0. 

#___________________________________________________________________________
# restructure edge_dxdy_l & edge_dxdy_r into edge_dx_lr0 & edge_dx_lr1
mdiag = mdiag.assign(edge_dx_lr=xr.DataArray( np.array([ mdiag.edge_dxdy_l[0,:], 
                                                         mdiag.edge_dxdy_r[0,:]]), dims=['n2', 'edg_n']).chunk(set_mdiag_chunk))
# dx & dy of right triangle --> norm vector 
mdiag = mdiag.assign(edge_dy_lr=xr.DataArray( np.array([ mdiag.edge_dxdy_l[1,:], 
                                                         mdiag.edge_dxdy_r[1,:]]), dims=['n2', 'edg_n']).chunk(set_mdiag_chunk))

#___________________________________________________________________________
# now drop edge_cross_dxdy variable dont need anymore
mdiag = mdiag.drop_vars(['edge_cross_dxdy', 'edge_dxdy_l', 'edge_dxdy_r']) #
mdiag = mdiag.load()

#___________________________________________________________________________
warnings.resetwarnings()
print(mdiag)
print(' --> elasped time: {} min.'.format( (clock.time()-ts)/60  ))
print(' --> mdiag uses {:3.2f} Gb:'.format(mdiag.nbytes/(1024**3)))

<xarray.Dataset>
Dimensions:     (n2: 2, edg_n: 371644)
Dimensions without coordinates: n2, edg_n
Data variables:
    edges       (n2, edg_n) int32 0 0 0 1 1 ... 126830 126832 126856 126857
    edge_tri    (n2, edg_n) int32 0 3 1 2 6 5 4 7 6 ... -1 -1 -1 -1 -1 -1 -1 -1
    edge_x      (n2, edg_n) float32 -179.7 -179.7 -179.7 ... 175.3 178.5 179.4
    edge_y      (n2, edg_n) float32 -77.94 -77.94 -77.94 ... -77.98 -78.02
    edge_mx     (edg_n) float32 -0.2757 0.1135 -179.4 ... 175.0 178.2 179.0
    edge_my     (edg_n) float32 -77.88 -77.78 -77.84 ... -77.49 -77.9 -78.0
    edge_dx_lr  (n2, edg_n) float32 -2.484e+03 -7.361e+03 -4.272e+03 ... 0.0 0.0
    edge_dy_lr  (n2, edg_n) float32 -5.209e+03 -867.5 8.242e+03 ... 0.0 0.0 0.0
 --> elasped time: 0.038156028588612875 min.
 --> mdiag uses 0.02 Gb:


In [ ]:
#___LOAD FESOM2 DATA___________________________________________________________________________________
# load t*u and t*v together
hflx_list = list()
for datapath, descript in zip(input_paths, input_names): 
    print(datapath, descript)
    ts = clock.time()
    if use_advflx:
        #__________________________________________________________________________________________________
        data = tpv.load_data_fesom2(mesh, datapath, vname='vec+tu+tv', year=year, descript=descript, 
                                    do_info=False, do_vecrot=False, do_zarithm='None', do_ie2n=False,
                                    chunks=chunks, do_compute=False, do_load=True, do_persist=False, )
        
        #__________________________________________________________________________________________________    
        # check if data where loaded
        if data is None: raise ValueError(f'data == None, data could not be readed, your path:{datapath} might be wrong!!!')

        #__________________________________________________________________________________________________
        # combine edge diagnostic and u,v data on edge_tri indices --> now dataset in context of edge
        # information --> to reindex here towards edges the data need to be unchunked, therefor you putted
        # do_load=True
        data = xr.merge([mdiag, data.tu[mdiag.edge_tri,:], data.tv[mdiag.edge_tri,:]])
        
    else:
        #__________________________________________________________________________________________________
        # load velocities
        data = tpv.load_data_fesom2(mesh, datapath, vname='vec+u+v', year=year, descript=descript, 
                                    do_info=False, do_vecrot=False, do_zarithm='None', do_ie2n=False, chunks=chunks, 
                                    do_compute=False, do_load=True, do_persist=False, )
        
        #__________________________________________________________________________________________________    
        # check if data where loaded
        if data is None: raise ValueError(f'data == None, data could not be readed, your path:{datapath} might be wrong!!!')
    
        #__________________________________________________________________________________________________    
        if use_bolusv:
            data['u'].data = data['u'].data + tpv.load_data_fesom2(mesh, datapath, vname='bolus_u', year=year, descript=descript, 
                                                                   do_info=False, do_vecrot=False, do_zarithm='None', do_ie2n=False,
                                                                   chunks=chunks, do_compute=False, do_load=True, do_persist=False)['bolus_u'].data
            data['v'].data = data['v'].data + tpv.load_data_fesom2(mesh, datapath, vname='bolus_v', year=year, descript=descript, 
                                                                   do_info=False, do_vecrot=False, do_zarithm='None', do_ie2n=False,
                                                                   chunks=chunks, do_compute=False, do_load=True, do_persist=False)['bolus_v'].data
        
        #__________________________________________________________________________________________________
        # load temperature
        datat = tpv.load_data_fesom2(mesh, datapath, vname='temp', year=year, descript=descript, 
                                     do_info=False, do_vecrot=False, do_zarithm='None', do_ie2n=False,
                                     chunks=chunks, do_compute=False, do_load=True, do_persist=False)
        # kick everything out that is not needed anymore
        datat = datat.drop_vars(list(datat.coords))
        
        #__________________________________________________________________________________________________
        # combine edge diagnostic and u,v data on edge_tri indices --> now dataset in context of edge
        # information --> to reindex here towards edges the data need to be unchunked, therefor you putted
        # do_load=True
        data  = xr.merge([mdiag, data.u[mdiag.edge_tri,:], data.v[mdiag.edge_tri,:], datat.temp[mdiag.edges,:]])
        del(datat)

    # kick everything out that is not needed anymore,
    # keep in mind that lon,lat here is the lon,lat position of the left and right triangle !!!
    data  = data.drop_vars(['elemi', 'elemiz', 'w_A', 'nz1', 'nzi', 'lon', 'lat', 'edge_mx']) #'edge_tri', 'edges', 
        
    # add vertical layer thickness as coordinate 
    data  = data.assign_coords(dz = xr.DataArray(np.diff(-mesh.zlev).astype('float32'), dims='nz1'))
    data  = data.set_coords(['edge_y', 'edge_dx_lr', 'edge_dy_lr', 'edge_tri', 'edge_my', 'edge_x', 'edges'])
        
    #__________________________________________________________________________________________________
    print(' --> elapsed time to load, prepare u,v, temp data: {:3.2f} min.'.format( (clock.time()-ts)/60  ))        
    print(' --> data_uvt uses {:3.2f} Gb:'.format(data.nbytes/(1024**3)))

    #__________________________________________________________________________________________________
    # compute meridional heat flux from t*v by binning
    ts = clock.time()
    hflx_list.append(tpv.calc_mhflx_box_fast(mesh, data, box, dlat=1.0, do_parallel=do_parallel, n_workers=parallel_nprc_bin, do_info=False))
    print(' --> elapsed time to comp. merid. hflux: {:3.2f} min.'.format( (clock.time()-ts)/60  ))   
    del(data)

/albedo/work/projects/p_fesom/pscholz/project_TRR181/trr181_tke_ctrl_ck0.1/5/ TKE
 --> elapsed time to load, prepare u,v, temp data: 0.35 min.
 --> data_uvt uses 0.28 Gb:


In [ ]:
#___PLOT FESOM2 DATA___________________________________________________________________________________
spath  = save_path
sname  = list(hflx_list[0][0].keys())[0]
slabel = hflx_list[0][0][sname].attrs['str_lsave']
if spath is not None: spath  = os.path.join(spath,'{}_{}_{}.png'.format(which_mode, sname, slabel)) 
if save_fname is not None: spath = save_fname
fig, ax = tpv.plot_mhflx(hflx_list, input_names, sect_name=None, figsize=[7*2, 3.5*2], 
                         do_allcycl=False, do_save=spath, save_dpi=which_dpi,)